# Primer examen parcial: parte práctica.
**Fecha**: 28 de Febrero de 2023.<br>
**Materia**: Herramientas Computacionales Avanzadas. <br>
**Profesor**: Luis Miguel de la Cruz Salas. <br>
**Escuela Nacional de Ciencias de la Tierra, UNAM**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import ipywidgets as widgets
import macti.visual as mvis
from macti.evaluacion import Quizz
quizz = Quizz('Derivada', 'DerivadasNumericas')

El modelo matemático para la conducción de calor en 1D con condiciones de frontera de tipo Dirichlet, con $\kappa$ = constante se escribe como sigue:

$$
\begin{eqnarray}
-\kappa \frac{d^2T}{dx^2} & = & S \; \text{ para } x \in [0,L] \\
T(x=0) & = & T_A \\
T(x=L) & = & T_B
\end{eqnarray}
$$

La solución analítica de este modelo matemático se escribe como sigue:

$$
T(x) =
\left(\frac{T_B - T_A}{L} + \frac{S}{2\kappa} \left(L - x\right) \right)x + T_A \tag{1}
$$

---
<a name='ej-1'></a>
### **<font color="DodgerBlue">Ejercicio 1.</font>**

<font color="DarkBlue">En la siguiente celda complete el código para implementar la fórmula $(1)$. </font>

---

In [ ]:
# Solucion exacta
def sol_exacta(x, TA, TB, S, L, k):
    """
    Calcula la temperatura usando la fórmula obtenida con Series de Taylor.

    Parameters
    ----------
    x: np.array
    Coordenadas donde se calcula la temperatura.

    TA: float
    Es la condición de frontera a la izquierda.
    
    TB: float
    Es la condición de frontera a la derecha.

    S: float
    es la fuente.
    
    L: float
    L es la longitud del dominio.
    
    k: float
    es la conductividad del material.
    
    Return
    ------
    al final esta función dibuja la solución.
    """
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
x=np.linspace(0,1,10)
TA = 1.0
TB = 0.0
S = 1.0
L = 1.0
k = 1.0

# Cálculo de la solución exacta.
T1 = sol_exacta(x, TA, TB, S, L, k)
print(T1)

In [ ]:
quizz.verifica('3', '1', T1)

El error absoluto y el error relativo se definen como sigue.</font>

$$
\begin{eqnarray*}
Error_{absoluto} & = & ||v_e - v_a|| \\ \\
Error_{relativo} & = & \dfrac{||v_e - v_a||}{||v_e||}
\end{eqnarray*}
$$

donde $v_e$ es el valor exacto y $v_a$ es el valor aproximado.

La siguiente función implementa la fórmula del $Error_{absoluto}$.

In [ ]:
def error_absoluto(ve, va):
    """
    Calcula el error absoluto entre el valor exacto (ve) y el valor aproximado (va).
    """
    return np.linalg.norm(ve - va)

---
<a name='ej-2'></a>
### **<font color="DodgerBlue">Ejercicio 2.</font>**

<font color="DarkBlue">En la siguiente celda complete el código para implementar la fórmula del $Error_{relativo}$. </font>

---

In [ ]:
def error_relativo(ve, va):
    """
    Calcula el error absoluto entre el valor exacto (ve) y el valor aproximado (va).
    """
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
datos=pd.DataFrame(np.array([[0.3000e+1,0.3100e+1],
                             [0.3000e-3,0.3100e-3],
                             [0.3000e+4,0.3100e+4]]),
                   columns=['pe','pa'])
datos['Error absoluto'] = np.array([error_absoluto(datos.pe[0], datos.pa[0]),
                error_absoluto(datos.pe[1], datos.pa[1]),
                error_absoluto(datos.pe[2], datos.pa[2])])
datos['Error relativo'] = np.array([error_relativo(datos.pe[0], datos.pa[0]),
                error_relativo(datos.pe[1], datos.pa[1]),
                error_relativo(datos.pe[2], datos.pa[2])])
datos

In [ ]:
quizz.verifica('3','2',np.array(datos['Error absoluto']))

In [ ]:
quizz.verifica('3','3',np.array(datos['Error relativo']))

---
<a name='ej-3'></a>
### **<font color="DodgerBlue">Ejercicio 3.</font>**

<font color="DarkBlue">El código de la siguiente celda, resuelve el problema de conducción de calor en 1D, como se revisó en clase. Para que esta celda de código funcione correctamente, deberá haber realizado los ejercicios 1 y 2. Además, en el código siguiente, deberá agregar dos líneas</font>

1. Para calcular la solución exacta y almacenarla en un arreglo de nombre `Te`. **Hint**. Revise en el ejercicio 1, cómo se calcula la solución exacta.
2. Para calcular el error absoluto y almacenarlo con el nombre `ea`. **Hint**. Revise en el código de abajo, cómo se calcula el error relativo.

---

In [ ]:
def conduccion_1d(k, S, L, TA, TB, N):
    """
    Calcula la temperatura en 1D mediante diferencias finitas.
    
    Parameters
    ----------    
    L: float
    L es la longitud del dominio.
    
    k: float
    es la conductividad del material.
    
    S: float
    es la fuente.
    
    TA: float
    Es la condición de frontera a la izquierda.
    
    TB: float
    Es la condición de frontera a la derecha.

    N: int
    Es el número de nodos internos (grados de libertad).
    
    Return
    ------
    al final esta función dibuja la solución.
    """

    # Cálculo de algunos parámetros numéricos
    h = L / (N+1)
    r = k / h**2
    
    # Definición de arreglos 
    T = np.zeros(N+2)
    b = np.zeros(N)
    A = np.zeros((N,N))

    # Se inicializa todo el arreglo b con S/r
    b[:] = S / r

    # Condiciones de frontera en el arreglo de la Temperatura.
    T[0] = TA
    T[-1] = TB
    
    # Se ajusta el vector del lado derecho (RHS) con las condiciones de frontera.
    b[0] += TA
    b[-1] += TB

    # Se calculan las entradas de la matriz del sistema de ecuaciones lineales.
    A[0,0] = 2
    A[0,1] = -1
    for i in range(1,N-1):
        A[i,i] = 2
        A[i,i+1] = -1
        A[i,i-1] = -1
    A[-1,-2] = -1
    A[-1,-1] = 2

    # Se resuelve el sistema lineal.
    T[1:N+1] = np.linalg.solve(A,b)

    # Coordenadas para la solución exacta.
    xe = np.linspace(0,L,100)
    
    # Coordenadas para la solución numérica.
    xa = np.linspace(0,L,N+2)
    
    # Se calcula la solución exacta en las coordenadas xe.
    # Te = ...
    
    # Se calcula el error absoluto.
    # ea = ...
    
    # YOUR CODE HERE
    raise NotImplementedError()
    
    # Se calcula el error relativo
    er = error_relativo(T, sol_exacta(xa,TA,TB,S,L,k))

    # Se imprime el error absoluto y el relativo.
    print('Error absoluto = {:6.5e}, Error relativo = {:6.5e}'.format(ea, er))

    # Se realiza la gráfica de la solución.
    plt.plot(xa, T, 'o-', lw = 1, label = 'Numérica', zorder=5)
    plt.plot(xe, Te, lw=5, label = 'Exacta')
    plt.xlabel('$x$')
    plt.ylabel('$T$')
    plt.legend()
    plt.show()
    
# Construcción del interactivo.
widgets.interactive(conduccion_1d,
                    k = widgets.FloatSlider(max=1.0, min=0.02, value=0.02, step=0.1), 
                    S = widgets.FloatSlider(max=10.0, min=0.0, value=0, step=1.0), 
                    L = widgets.fixed(5.0), 
                    TA = widgets.fixed(200), 
                    TB = widgets.fixed(1000), 
                    N = widgets.IntSlider(max=10, min=4, value=4))